# Machine Learning - Wine Data

In [2]:
import pandas as pd
import re, datetime
import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop = stopwords.words('english')
import string

### Basic Cleaning

In [24]:
all_wines = pd.read_csv('../Data/winemag-data-130k-v2.csv')

Dropping columns we don't need.

In [25]:
clean_wines = all_wines.drop(columns=['Unnamed: 0', 'taster_twitter_handle', 'region_2'])
clean_wines = clean_wines.dropna()

Replacing/translating wines from other countries into English origin.

In [26]:
clean_wines['variety'] = clean_wines['variety'].replace(['Weissburgunder'], 'Chardonnay')
clean_wines['variety'] = clean_wines['variety'].replace(['Spatburgunder'], 'Pinot Noir')
clean_wines['variety'] = clean_wines['variety'].replace(['Grauburgunder'], 'Pinot Gris')
clean_wines['variety'] = clean_wines['variety'].replace(['Garnacha'], 'Grenache')
clean_wines['variety'] = clean_wines['variety'].replace(['Pinot Nero'], 'Pinot Noir')
clean_wines['variety'] = clean_wines['variety'].replace(['Alvarinho'], 'Albarino')

Extracting the vintage from the title, creating a new column for the year, then dropping the title column due to redundant information.

In [27]:
clean_wines['vintage'] = clean_wines['title'].str.extract('(\d\d\d\d)', expand=True)
clean_wines = clean_wines.drop(columns='title')

Creating new column for type of wine (i.e., white and red).

In [28]:
clean_wines['type'] = ['white' if x in ['Chardonnay', 'Riesling', 'Sauvignon Blanc', 'White Blend', 'Sparkling Blend', 'Pinot Gris',
                                        'Champagne Blend', 'GrÃ¼ner Veltliner', 'Pinot Grigio', 'Portuguese White', 'Viognier',
                                        'GewÃ¼rztraminer', 'GewÃ¼rztraminer'] else 'red' for x in clean_wines['variety']]

Creating a new column for word count in the description, may potentially see some correlation down the road. 

In [29]:
clean_wines['word count'] = clean_wines['description'].str.lower().str.split().str.len()

Creating a new feature to easily feed the model, a column that describes the age of each wine. 

In [30]:
clean_wines = clean_wines[clean_wines['vintage'].notna()]
clean_wines = clean_wines.copy()
clean_wines['vintage'] = clean_wines['vintage'].astype(int)
clean_wines['age'] = 2020 - clean_wines['vintage'] 

Dropping duplicate rows.

In [31]:
clean_wines = clean_wines.drop_duplicates('description')
clean_wines = clean_wines[pd.notnull(clean_wines.price)]

Tokenizing the description column to break apart the sentence into individual words, which will then be parsed.

In [32]:
clean_wines['tokenized_text'] = clean_wines['description'].apply(word_tokenize) 

NLTK has a problem reading stop words which are upper-case, so we make sure all of the words in this column lower-case to make sure we get as many as we can.

In [33]:
clean_wines['tokenized_text'] = clean_wines['tokenized_text'].apply(lambda x: [w.lower() for w in x])

We remove stop words here.

In [34]:
stop.append("'s")

In [35]:
clean_wines['tokenized_text'] = clean_wines['tokenized_text'].apply(lambda x: [item for item in x if item not in stop])

Resetting the index to help with the merge.

In [36]:
clean_wines = clean_wines.reset_index(drop=True)

Creating a separate dataframe that explodes each individual string in the list to a separate column.

In [37]:
descriptors = pd.DataFrame(clean_wines.tokenized_text.values.tolist()).add_prefix('desc_')

Resetting the index to help with the merge.

In [38]:
descriptors = descriptors.reset_index(drop=True)

Merging the two dataframes on the reset index.

In [39]:
expanded_df = pd.merge(clean_wines, descriptors, left_index=True, right_index=True)

Dropping the tokenized and description columns.

In [40]:
expanded_df = expanded_df.drop(columns=['tokenized_text', 'description'])

Through the tokenization and splitting the strings, I have a ton of leftover punctuation. This strips that from the dataframe.

In [41]:
expanded_df

,country,designation,points,price,province,region_1,taster_name,variety,winery,vintage,...,desc_74,desc_75,desc_76,desc_77,desc_78,desc_79,desc_80,desc_81,desc_82,desc_83
0,US,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,Riesling,St. Julian,2013,...,None,None,None,None,None,None,None,None,None,None
1,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Noir,Sweet Cheeks,2012,...,None,None,None,None,None,None,None,None,None,None
2,Spain,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tempranillo-Merlot,Tandem,2011,...,None,None,None,None,None,None,None,None,None,None
3,Italy,Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Frappato,Terre di Giurfo,2013,...,None,None,None,None,None,None,None,None,None,None
4,France,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Pinot Gris,Jean-Baptiste Adam,2012,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47139,France,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Pinot Gris,Cave de Turckheim,2010,...,None,None,None,None,None,None,None,None,None,None
47140,Italy,Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Nero d'Avola,Cusumano,2012,...,None,None,None,None,None,None,None,None,None,None
47141,France,Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Pinot Gris,Domaine Rieflé-Landmann,2013,...,None,None,None,None,None,None,None,None,None,None
47142,France,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Gewürztraminer,Domaine Gresser,2013,...,None,None,None,None,None,None,None,None,None,None


In [42]:
expanded_df = expanded_df.replace('['+string.punctuation+']', '', regex=True)

Exporting to CSV to use in Tableau.

In [43]:
#clean_wines.to_csv('../Data/cleaned_wine_data.csv',index=False)
expanded_df.to_csv('../Data/expanded_wine_data.csv',index=False)

Converting string data into numerical data.

In [13]:
binary_df = pd.read_csv('../Data/text_predictions_binary.csv')

In [14]:
binary_df = binary_df.drop(columns='Unnamed: 0')

In [15]:
binary_df

,prediction,actual
0,Bad,Bad
1,Good,Good
2,Good,Bad
3,Good,Good
4,Good,Good
...,...,...
35982,Good,Bad
35983,Good,Good
35984,Bad,Good
35985,Bad,Bad


In [17]:
binary_df['pred_numeric'] = binary_df['prediction']
binary_df['pred_actual'] = binary_df['actual']
binary_df

,prediction,actual,pred_numeric,pred_actual
0,Bad,Bad,Bad,Bad
1,Good,Good,Good,Good
2,Good,Bad,Good,Bad
3,Good,Good,Good,Good
4,Good,Good,Good,Good
...,...,...,...,...
35982,Good,Bad,Good,Bad
35983,Good,Good,Good,Good
35984,Bad,Good,Bad,Good
35985,Bad,Bad,Bad,Bad


In [19]:
binary_df['pred_numeric'] = binary_df['pred_numeric'].replace({'Bad': 0, 'Good': 1})
binary_df['pred_actual'] = binary_df['pred_actual'].replace({'Bad': 0, 'Good': 1})


TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [20]:
binary_df

,prediction,actual,pred_numeric,pred_actual
0,Bad,Bad,0,0
1,Good,Good,1,1
2,Good,Bad,1,0
3,Good,Good,1,1
4,Good,Good,1,1
...,...,...,...,...
35982,Good,Bad,1,0
35983,Good,Good,1,1
35984,Bad,Good,0,1
35985,Bad,Bad,0,0


In [21]:
binary_df.to_csv('../Data/text_predictions_binary_int.csv',index=True)